In [61]:
import numpy as np
import matplotlib.pyplot as plt
from emnist import extract_training_samples
from functools import partial

from vertices_generator import vertices
from kernel import Gaussian_kernel
from mdwsvm_ad import mdwsvm_ad
from mdwsvm import mdwsvm
from one_class_svm import one_class_svm
from hybrid import hybrid
from metric import within_class_error

In [62]:
# Load data
digits_images, digits_labels = extract_training_samples('digits')
letters_images, letters_labels = extract_training_samples('byclass')

# Get number 0, 5, 7, 8
mask_0 = (digits_labels == 0)
digits_images_0 = digits_images[mask_0]
digits_labels_0 = digits_labels[mask_0]

mask_5 = (digits_labels == 5)
digits_images_5 = digits_images[mask_5]
digits_labels_5 = digits_labels[mask_5]

mask_7 = (digits_labels == 7)
digits_images_7 = digits_images[mask_7]
digits_labels_7 = digits_labels[mask_7]

mask_8 = (digits_labels == 8)
digits_images_8 = digits_images[mask_8]
digits_labels_8 = digits_labels[mask_8]


# Get letter v, w, y
mask_vwy = (letters_labels == 57) | (letters_labels == 58) | (letters_labels == 60)
letters_images = letters_images[mask_vwy]
letters_labels = letters_labels[mask_vwy]
print(len(letters_labels))
#plt.imshow(letters_images[3,:,:])

7972


In [63]:
# Get training and testing data
X_train = np.zeros((4000,28,28))
y_train = np.zeros((4000), dtype=int)
X_test = np.zeros((8000,28,28))
y_test = np.zeros((8000), dtype=int)

# 4000 digits normalized training data 
X_train[0:1000,:,:] = digits_images_0[0:1000,:,:] / 255
X_train[1000:2000,:,:] = digits_images_5[0:1000,:,:] / 255
X_train[2000:3000,:,:] = digits_images_7[0:1000,:,:] / 255
X_train[3000:4000,:,:] = digits_images_8[0:1000,:,:] / 255
X_train = X_train.reshape(4000,784).T 
# 4000 digits training label
y_train[0:1000] = digits_labels_0[0:1000]
y_train[1000:2000] = digits_labels_5[0:1000]
y_train[2000:3000] = digits_labels_7[0:1000]
y_train[3000:4000] = digits_labels_8[0:1000]

# Get 400 digits for test X
X_test[0:100,:,:] = digits_images_0[1000:1100,:,:] / 255
X_test[100:200,:,:] = digits_images_5[1000:1100,:,:] / 255
X_test[200:300,:,:] = digits_images_7[1000:1100,:,:] / 255
X_test[300:400,:,:] = digits_images_8[1000:1100,:,:] / 255
# 400 digits testing label
y_test[0:100] = digits_labels_0[1000:1100]
y_test[100:200] = digits_labels_5[1000:1100]
y_test[200:300] = digits_labels_7[1000:1100]
y_test[300:400] = digits_labels_8[1000:1100]
# Get 7600 lowercase letters
X_test[400:8000,:,:] = letters_images[0:7600,:,:] / 255
y_test[400:8000] = letters_labels[0:7600]

# 2000 digits and 38000 letters normalized data, 0-9 are 0-9, 36-61 are a-z
X_test = X_test.reshape(8000,784).T

In [74]:
err_svm = np.zeros((6,6)) # 6 sigma value, 6 v value
sig_values = [0.001, 0.01, 0.1, 1, 10, 100]
v_values = [0.001, 0.01, 0.1, 0.3, 0.6, 0.9]
# True symbol for testing data
y_true = np.ones((8000), dtype=int)
y_true[400:8000] = np.full((7600), -1, dtype=int)

# Initialize
best_v = 0
best_sig = 0
best_score = -1
i = 0
j = 0
# Loop to find best v and sig value
for sig in sig_values:
    k = partial(Gaussian_kernel, sigma2 = sig)
    
    for v in v_values:
        model = one_class_svm(X_test, v = v, K = k)
        y_pred = model.predict()
        score = 1 - within_class_error(y_ture = y_true, y_pred = y_pred)
        
        # Check if the current value is the best so far
        if score > best_score:
                best_v = v
                best_sig = sig
                best_score = score
                
        # Record the current error and all the parameter values
        err_svm[i,j] = score
        j += 1
    # Set index
    i += 1
    j = 0


In [ ]:
print(best_v)
print(best_sig)
print(best_score)

In [ ]:
# Plot
fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(8, 3))

axs[0].set_ylabel('Error')
axs[0].set_title('v')
for i in range(6):
    axs[0].plot(v_values, err_svm[i,:], color = 'black', linestyle = '-', label = 'v')

axs[1].set_title('sig')
for i in range(6):
    axs[1].plot(sig_values, err_svm[:,i], color = 'black', linestyle = '-', label = 'sig')

plt.tight_layout()
plt.show()


NameError: name 'y_test_2' is not defined